In [ ]:
import pypsa
import yaml
import cartopy
import sys
import re

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from itertools import product
from matplotlib.lines import Line2D
from vresutils.costdata import annuity

path = "../../../playgrounds/pr/pypsa-eur-sec/"

sys.path.append(path + "scripts/")
from plot_summary import rename_techs#, preferred_order

plt.style.use(['bmh', '../matplotlibrc'])
xr.set_options(display_style='html')

%matplotlib inline

In [ ]:
nodes = 181
opt = "Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10"
output = "../results/graphics/"

In [ ]:
with open(path + "config.yaml") as file:
    config = yaml.safe_load(file)

In [ ]:
def rename_techs_tyndp(tech):
    tech = rename_techs(tech)
    if "heat pump" in tech or "resistive heater" in tech:
        return "power-to-heat"
    elif tech in ["H2 Electrolysis", "methanation", "helmeth", "H2 liquefaction"]:
        return "power-to-gas"
    elif tech == "H2":
        return "H2 storage"
    elif tech in ["OCGT", "CHP", "gas boiler", "H2 Fuel Cell"]:
        return "gas-to-power/heat"
    elif "solar" in tech:
        return "solar"
    elif tech == "Fischer-Tropsch":
        return "power-to-liquid"
    elif "offshore wind" in tech:
        return "offshore wind"
    elif "CC" in tech or "sequestration" in tech:
        return "CCS"
    else:
        return tech

In [ ]:
preferred_order = pd.Index([
    'transmission lines',
    'electricity distribution grid',
    'hydroelectricity',
    'hydro reservoir',
    'run of river',
    'pumped hydro storage',
    'solid biomass',
    'biogas',
    'onshore wind',
    'offshore wind',
    'offshore wind (AC)',
    'offshore wind (DC)',
    'solar PV',
    'solar thermal',
    'solar rooftop',
    'solar',
    'building retrofitting'
    'ground heat pump',
    'air heat pump',
    'heat pump',
    'resistive heater',
    'power-to-heat',
    'gas-to-power/heat',
    'CHP',
    'OCGT',
    'gas boiler',
    'gas',
    'natural gas',
    'helmeth',
    'methanation',
    'power-to-gas',
    'power-to-H2',
    'H2 pipeline',
    'H2 liquefaction',
    'H2 storage',
    'hydrogen storage',
    'power-to-liquid',
    'battery storage',
    'hot water storage',
    'CO2 sequestration',
    'CCS'
])

## Line Volume Sensitivity

In [ ]:
folder = "results/20211007-181-lv"

In [ ]:
costs = pd.read_csv(path + folder + "/csvs/costs.csv", header=[0,1,2,3], index_col=[0,1,2])

In [ ]:
costs = costs[str(nodes)].rename(lambda x: float(x), axis=1, level=0)

In [ ]:
df = costs.xs(opt, level='opt', axis=1).xs('2030', level="planning_horizon", axis=1).groupby(level=2).sum().div(1e9)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
(1 - df.sum() / df.sum().max()) * 100

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.) < 1.2]
to_drop = to_drop.difference(["DAC", "hot water storage"]) # exclude dropping to align with onwind sensitivity...
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
df = df.loc[order]

In [ ]:
fig, ax = plt.subplots(figsize=(6,4.5))

to_plot = df.T.sort_index()

tech_colors = config['plotting']['tech_colors']
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(
    ax=ax,
    stacked=True,
    linewidth=0,
    color=colors
)

handles,labels = ax.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax.set_xlim(1,2)
ax.set_xlabel("Line volume limit (multiple of today's volume)")

ax.set_ylim([0,1000])
ax.set_ylabel("System Cost [EUR billion per year]")
ax.grid(axis="y")

ax.axvline(
    1.25,
    color='k',
    linestyle="--",
    linewidth=1
)

ax.text(1.275, 810, "TYNDP equivalent", size=13, color="k")

# legend on side
#ax.legend(handles, labels, ncol=1, frameon=False, bbox_to_anchor=(1,1.05))

# legend inside
ax.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1,1.48))

ax.set_axisbelow(False)

fig.savefig(output + "lv-sensitivity.pdf", bbox_inches='tight')

## Onshore Wind Sensitivity

In [ ]:
folder = "results/20211007-181-onw"

In [ ]:
costs = pd.read_csv(path + folder + "/csvs/costs.csv", header=[0,1,2,3], index_col=[0,1,2])

In [ ]:
costs = costs.xs([str(nodes), "1.0", "2030"], level=["cluster", "lv", "planning_horizon"], axis=1)

In [ ]:
costs.columns = [100 * float(re.search(r'onwind\+p([0-9.]*)', c).groups()[0]) for c in costs.columns]

In [ ]:
df = costs.groupby(level=2).sum().div(1e9)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum()

In [ ]:
df.sum() - df.sum().max()

In [ ]:
(1 - df.sum() / df.sum().max()) * 100

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.) < 1.2]
print(to_drop)
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
df = df.loc[order]

In [ ]:
fig, ax = plt.subplots(figsize=(6,4.5))

to_plot = df.T.sort_index()

tech_colors = config['plotting']['tech_colors']
colors = [tech_colors[i] for i in df.index]

to_plot.plot.area(
    ax=ax,
    stacked=True,
    linewidth=0,
    color=colors
)

handles,labels = ax.get_legend_handles_labels()

handles.reverse()
labels.reverse()

ax.set_xlim(0,100)
ax.set_xlabel("Fraction of technical onshore wind potential available [%]")

ax.set_ylim(0,1000)
ax.set_ylabel("System Cost [EUR billion per year]")
ax.grid(axis="y")

ax.axvline(
    25,
    color='k',
    linestyle="--",
    linewidth=1
)

ax.text(27.5, 900, "compromise social potential", size=13, color="k")

# legend on side
#ax.legend(handles, labels, ncol=1, frameon=False, bbox_to_anchor=(1,1.05))

# legend inside
ax.legend(handles, labels, ncol=3, frameon=False, bbox_to_anchor=(1.1,1.48))   

ax.set_axisbelow(False)

fig.savefig(output + "onw-sensitivity.pdf", bbox_inches='tight')

## H2 Network Scenarios

In [ ]:
folder = "results/20211007-181-h2"

In [ ]:
costs = pd.read_csv(path + folder + "/csvs/costs.csv", header=[0,1,2,3], index_col=[0,1,2])

In [ ]:
costs = costs.xs([str(nodes), "2030"], level=["cluster", "planning_horizon"], axis=1)

In [ ]:
def parse_index(c):
    
    lv = c[0]

    match = re.search(r'onwind\+p([0-9.]*)', c[1])
    onw = 100. if match is None else 100 * float(match.groups()[0])

    h2 = "no H2 grid" if "noH2network" in c[1] else "H2 grid"
    
    return (lv, onw, h2)

In [ ]:
costs.columns = pd.MultiIndex.from_tuples([parse_index(c) for c in costs.columns], names=["lv", "onw", "h2"])

In [ ]:
df = costs.groupby(level=2).sum().div(1e9)

In [ ]:
df = df.groupby(df.index.map(rename_techs_tyndp)).sum()

In [ ]:
df.sum().sort_values()

In [ ]:
dff = df.sum().unstack("h2")

In [ ]:
rel_advantage = (((dff["no H2 grid"] / dff["H2 grid"] - 1) * 100).round(1)).unstack().T
rel_advantage

In [ ]:
abs_advantage = ((dff["no H2 grid"] - dff["H2 grid"]).round().astype(int)).unstack().T
abs_advantage

In [ ]:
(df.sum() - df.sum().max()).sort_values().round()

In [ ]:
((1 - df.sum() / df.sum().max()) * 100).sort_values().round()

In [ ]:
x = df.sum().xs(100., level='onw')
xx = x / x.min()
xx.unstack().round(2)

In [ ]:
to_drop = df.index[df.max(axis=1).fillna(0.) < 1.2]
print(to_drop)
df.drop(to_drop, inplace=True)

In [ ]:
order = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
df = df.loc[order]

In [ ]:
tech_colors = config['plotting']['tech_colors']
colors = [tech_colors[i] for i in df.index]

In [ ]:
xx = enumerate(df.columns.get_level_values('lv').unique()[::-1])
yy = enumerate(df.columns.get_level_values('onw').unique())

fig, axs = plt.subplots(3, 3, figsize=(6.5,6.5), sharex=True, sharey=True)

kwargs = dict(
    stacked=True,
    color=colors,
    ylim=(0,1000),
    legend=False
)

for x, y in product(xx, yy):
    
    ax = axs[x[0], y[0]]
    
    toprow_kwargs = dict(
        title=f"onshore wind\n{int(y[1])}% potential\n"
    ) if x[0] == 0 else {}
    
    ylabel_value = "optimal" if x[1] == "opt" else f"{100 * float(x[1]) - 100:.{0}f}%"

    to_plot = df.xs([x[1], y[1]], axis=1, level=["lv", "onw"]).T.sort_index(ascending=True)
    to_plot.plot.bar(ax=ax, **kwargs,
                     ylabel=f"line expansion\n{ylabel_value}\n\nbn€/a",
                     **toprow_kwargs)
    
    ax.set_xlabel('', rotation=0)
    
    ax.grid(axis='y')
    ax.title.set_size(11)
    
    #ax.patch.set_visible(False)
    
    for i in ['top', 'right', 'left', 'bottom']:
        ax.spines[i].set_visible(False)
    
    label = f"+ {rel_advantage.loc[y[1],x[1]]}%\n+ {abs_advantage.loc[y[1],x[1]]}"
    ax.text(0.28, 820, label, size=8.5, color="k")
    

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
fig.legend(handles, labels, bbox_to_anchor=(1.33,.9))

plt.tight_layout()

plt.savefig(output + "sensitivity-h2.pdf", bbox_inches='tight')

### H2 vs Electricity Grid

In [ ]:
levels = ["lv", "onw", "h2"]

In [ ]:
scenA = df.xs(["1.0", 100., "H2 grid"], level=levels, axis=1)
scenB = df.xs(["opt", 100., "no H2 grid"], level=levels, axis=1)


In [ ]:
scenA.columns = ["no electricity grid exp.\nH2 grid exp."]
scenB.columns = ["electricity grid exp.\nno H2 grid exp."]


In [ ]:
to_plot = pd.concat([scenB, scenA], axis=1).T

In [ ]:
tsc = to_plot.sum(axis=1)

In [ ]:
diff_rel = (100 * tsc / tsc.min() - 100).round(1)[1]

In [ ]:
diff_abs = (tsc - tsc.min()).round(1)[1]

In [ ]:
fig, ax = plt.subplots(figsize=(6.5,5.3))

to_plot.plot.bar(ax=ax, stacked=True, color=colors, ylim=(0,900), ylabel="total system cost [bn€/a]")

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1,1))

label = f"+ {diff_rel}%"
ax.text(0.28, 820, label, size=11, color="#444444")

label = f"+ {diff_abs} bn€/a"
ax.text(0.28, 860, label, size=11, color="k")

ax.grid(axis='y')
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0,901,100), fontsize=11)

for i in ['top', 'right', 'left', 'bottom']:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(output + "h2-vs-elec-grid.pdf", bbox_inches='tight')

### Growing Exclusion

In [ ]:
levels = ["lv", "onw", "h2"]

In [ ]:
scenA = df.xs(["opt", 100., "H2 grid"], level=levels, axis=1)
scenB = df.xs(["1.0", 100., "H2 grid"], level=levels, axis=1)
scenC = df.xs(["1.0", 0., "H2 grid"], level=levels, axis=1)
scenD = df.xs(["1.0", 0., "no H2 grid"], level=levels, axis=1)

In [ ]:
scenA.columns = ["least-cost"]
scenB.columns = ["no grid expansion"]
scenC.columns = ["no grid expansion\nno onshore wind"]
scenD.columns = ["no grid expansion\nno onshore wind\nno hydrogen grid"]

In [ ]:
to_plot = pd.concat([scenA, scenB, scenC, scenD], axis=1).T

In [ ]:
to_plot

In [ ]:
to_plot.loc["today", "today"] = 700

order = to_plot.index[:-1].insert(0, to_plot.index[-1])
to_plot = to_plot.loc[order]

In [ ]:
to_plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5.3))

to_plot.plot.bar(ax=ax, stacked=True, color=colors, ylim=(0,1000), ylabel="total system cost [bn€/a]")

handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
plt.legend(handles, labels, bbox_to_anchor=(1,1.05))

ax.grid(axis='y')
plt.xticks(rotation=0, fontsize=11)
plt.yticks(np.arange(0,1001,100), fontsize=11)

for i in ['top', 'right', 'left', 'bottom']:
    ax.spines[i].set_visible(False)

plt.tight_layout()

plt.savefig(output + "growing-exclusion.pdf", bbox_inches='tight')

### Unused

In [ ]:
costs4 = pd.DataFrame(costs3.sum()).T/1e9
costs4.index = ["system"]

costs4.loc["environment"] = [180*3.5,50.,50.,50.]

costs4.loc["health"] = [600.,50.,50.,50.]

costs4.loc["acceptance"] = [50.,400.,200.,0.]

costs4

In [ ]:
if True:


    fig, ax = plt.subplots()
    fig.set_size_inches(7,5)

    to_plot = costs4.T

    to_plot.plot(kind="bar",stacked=True,linewidth=0,ax=ax)
                    # color=[snakemake.config['plotting']['tech_colors'][i] for i in df.index])

    handles,labels = ax.get_legend_handles_labels()

    handles.reverse()
    labels.reverse()
    #ax.set_ylim([0,50])

    #ax.set_xlim([0,100])

    ax.set_ylabel("Total costs (schematic)")

    ax.set_xlabel("")

    #ax.grid(axis="y")
    
    plt.xticks(rotation=0)
    
    #ax.get_yaxis().set_visible(False)
    ax.get_yaxis().set_ticks([])

    #ax.legend().set_visible(False)


    #framealpha stops transparency                                                                                        
    #bbox: first is x, second is y
    
    #ax.plot([1.25,1.25],[2000,-10],color="r",linewidth=2,linestyle="--")

    #ax.text(1.26,930,"TYNDP equivalent",size="14",color="r")

    ax.legend(handles, labels, ncol=1,framealpha=1.)#loc="upper center",                       
    fig.tight_layout()

    fig.savefig("200629-solar3-bar-total.pdf",transparent=True)


In [ ]:
if True:


    fig, ax = plt.subplots()
    fig.set_size_inches(7,5)

    to_plot = costs4.T

    to_plot.plot(kind="bar",stacked=True,linewidth=0,ax=ax)
                    # color=[snakemake.config['plotting']['tech_colors'][i] for i in df.index])

    handles,labels = ax.get_legend_handles_labels()

    handles.reverse()
    labels.reverse()
    #ax.set_ylim([0,50])

    #ax.set_xlim([0,100])

    ax.set_ylabel("Total costs (schematic)")

    ax.set_xlabel("")

    #ax.grid(axis="y")
    
    plt.xticks(rotation=0)
    
    #ax.get_yaxis().set_visible(False)
    ax.get_yaxis().set_ticks([])

    #ax.legend().set_visible(False)


    #framealpha stops transparency                                                                                        
    #bbox: first is x, second is y
    
    #ax.plot([1.25,1.25],[2000,-10],color="r",linewidth=2,linestyle="--")

    #ax.text(1.26,930,"TYNDP equivalent",size="14",color="r")

    ax.legend(handles, labels, ncol=1,framealpha=1.)#loc="upper center",                       
    fig.tight_layout()

    fig.savefig("200629-solar3-bar-total.pdf",transparent=True)
